In [1]:

import sys 
import os
configfile = '~/SNAIL/symbolic/no.py'
sys.path.append(os.path.dirname(os.path.expanduser(configfile)))
from sympy.physics.secondquant import *
from sympy import symbols,I
import sympy as sp
from no import normal

In [2]:
ind_a = symbols('a')
a = B(ind_a)
ad = Bd(ind_a)

In [3]:
g3 = symbols('g3', real=True)
g4 = symbols('g4', real=True)
g5 = symbols('g5', real=True)
g6 = symbols('g6', real=True)
omega = symbols('omega', real=True)
kappa = symbols('kappa', real=True)
v = normal(g3 * (a+ad)**3 +g4*(a+ad)**4  )
h0 = omega * ad * a

In [4]:
v

3*g3*AnnihilateBoson(a) + g3*AnnihilateBoson(a)**3 + 3*g3*CreateBoson(a) + 3*g3*CreateBoson(a)*AnnihilateBoson(a)**2 + 3*g3*CreateBoson(a)**2*AnnihilateBoson(a) + g3*CreateBoson(a)**3 + 3*g4 + 6*g4*AnnihilateBoson(a)**2 + g4*AnnihilateBoson(a)**4 + 12*g4*CreateBoson(a)*AnnihilateBoson(a) + 4*g4*CreateBoson(a)*AnnihilateBoson(a)**3 + 6*g4*CreateBoson(a)**2 + 6*g4*CreateBoson(a)**2*AnnihilateBoson(a)**2 + 4*g4*CreateBoson(a)**3*AnnihilateBoson(a) + g4*CreateBoson(a)**4

In [5]:
v1 = sp.simplify(v).expand()
temp = 0
for term in v1.args:
    if term.as_powers_dict()[ad] == term.as_powers_dict()[a]:
        temp += term
h1 = temp * 1
temp = 0
for term in (v1-h1).args:
    if term.as_powers_dict()[ad]!= term.as_powers_dict()[a]:
        temp += term / omega / (term.as_powers_dict()[ad] - term.as_powers_dict()[a])
s1= normal(temp) * 1
s_tot = s1 * kappa 
s_tot_1 = s1 * kappa

In [6]:
h_tot = h0 + kappa*v 
h_eff_2 = 0

temp_2 = normal(Commutator(s_tot_1, h_tot.coeff(kappa,1)*kappa**1).doit() )
temp_1 = normal(Commutator(s_tot_1, h_tot.coeff(kappa,0)*kappa**0).doit() )
print('x')
h_eff_2 += temp_2

temp_2 = normal(Commutator(s_tot_1, temp_1).doit())
print('x')
h_eff_2 += sp.Rational(1,2) * temp_2

h_eff_2 = h_eff_2.expand()
v2 = normal(h_eff_2.coeff(kappa,2))
temp = 0
for term in v2.args:
    if term.as_powers_dict()[ad] == term.as_powers_dict()[a]:
        temp += term
h2 = temp * 1
temp = 0
for term in v1.args:
    if term.as_powers_dict()[ad] == term.as_powers_dict()[a]:
        temp += term
h1 = temp * 1
temp = 0
for term in (v2-h2).args:
    if term.as_powers_dict()[ad] != term.as_powers_dict()[a]:
        temp += term / omega / (term.as_powers_dict()[ad] - term.as_powers_dict()[a])
s2 = normal(temp) * 1


x
x


In [7]:
def remove_higher(expr):
    terms_higher = [
        term for term in expr.args if (term.has(g4**2) or term.has(g3**2*g4) or term.has(g3**4) or term.has(g3*g4) or term.has(g3**2) or term.has(g4))]
    return expr - sum(terms_higher)
def remove_cons(expr):
    terms_higher = [
        term for term in expr.args if (term.has(a) or term.has(ad)) ]
    
    return sum(terms_higher)

In [8]:
s1c = normal(Commutator(s1, ((a+ad))).doit() )
s2c = normal(Commutator(s2, ((a+ad))).doit() )+(sp.Rational(1,2) *normal(Commutator(s1, s1c).doit() )).expand()
s2c = remove_higher(s2c.expand())

In [9]:
def replace_scalars_with_valued_matrices(expr, matrix_values):
        """
        Replace scalar symbols and scalar operations in a SymPy scalar expression with valued matrices.

        Args:
        expr (sympy.Expr): The input scalar expression.
        matrix_values (dict): A dictionary mapping scalar symbols to valued matrices.

        Returns:
        sympy.Expr: The modified expression with valued matrices.
        """

        def map_operations(node):
            if (isinstance(node, AnnihilateBoson) or isinstance(node, CreateBoson) )and node in matrix_values:
                return matrix_values[node]
            elif isinstance(node, sp.Add):
                return sp.Add(*[map_operations(arg) for arg in node.args])
            elif isinstance(node, sp.Mul):
                return sp.Mul(*[map_operations(arg) for arg in node.args])
            elif isinstance(node, sp.Pow):
                base, exponent = map_operations(node.base), map_operations(node.exp)
                return base**exponent
            else:
                return node

        return map_operations(expr)

In [10]:
# Initialize the annihilation (lowering) and creation (raising) matrices with zeros
from sympy import symbols, sqrt
n=6
s = sp.zeros(n)
s_dagger = sp.zeros(n)

# Fill the matrices using sympy functions
for i in range(n-1):
    s[i, i+1] = sqrt(i+1)
    s_dagger[i+1, i] = sqrt(i+1)



In [11]:
matrix_values = {a: s, ad: s_dagger}
g_matrix = replace_scalars_with_valued_matrices(remove_cons((a+ad)+remove_higher(s1c+s2c).expand()).expand(), matrix_values)

In [12]:
Vs = g_matrix - 6*g3/omega*sp.eye(n)

In [13]:
import sympy as sp
omega_s = sp.Symbol('omega_s')
alpha = sp.Symbol('alpha')
# Generate the diagonal elements
diagonal = [i*omega_s + alpha*i*(i-1)/2 for i in range(0, n)]

# Create the diagonal matrix
Hs = sp.diag(*diagonal)
def generate_H1_H2(d):
    # Create symbolic variables for epsilon1 and epsilon2, but set the first element to 0
    epsilon1 = [0] + [sp.Symbol(f'epsilon^1_{i}') for i in range(1, d)]
    epsilon2 = [0] + [sp.Symbol(f'epsilon^2_{i}') for i in range(1, d)]

    # Generate H1 (diagonal matrix with epsilon^1_i, first element is 0)
    H1 = sp.diag(*epsilon1)

    # Generate H2 (diagonal matrix with epsilon^2_i, first element is 0)
    H2 = sp.diag(*epsilon2)

    return H1, H2


def generate_V1_V2(n):
    # Create symbolic variables for V1
    V1 = sp.zeros(n, n)
    for i in range(n):
        for j in range(n):
            if i != j:
                # Check if both i and j are either both even or both odd
                if (i % 2 == j % 2):
                    V1[i, j] = 0  # Set to 0 if both are odd or both are even
                else:
                    V1[i, j] = sp.Symbol(f'phi^1_{i}{j}')
                    V1[j, i] = V1[i, j]  # Ensure symmetry

    # Create symbolic variables for V2
    V2 = sp.zeros(n, n)
    for i in range(n):
        for j in range(n):
            if i != j:
                # Check if both i and j are either both even or both odd
                if (i % 2 == j % 2):
                    V2[i, j] = 0  # Set to 0 if both are odd or both are even
                else:
                    V2[i, j] = sp.Symbol(f'phi^2_{i}{j}')
                    V2[j, i] = V2[i, j]  # Ensure symmetry

    return V1, V2


# Set the dimension
d = 2

# Generate H1 and H2
H1, H2 = generate_H1_H2(d)
V1, V2 = generate_V1_V2(d)

Is = sp.eye(n)
I1 = I2 = sp.eye(d)


In [14]:
from sympy.physics.quantum import TensorProduct
EL = sp.Symbol('E_L')
phis = sp.Symbol('phi_s')
Hs = TensorProduct(Hs,TensorProduct(I2,I1))
H1 = TensorProduct(Is,TensorProduct(I2,H1))
H2 = TensorProduct(Is,TensorProduct(H2,I1))
H_int1 = -EL/2*phis*TensorProduct(Vs,TensorProduct(I2,V1))
H_int2 = EL/2*phis*TensorProduct(Vs,TensorProduct(V2,I1))


In [15]:
V1

Matrix([
[       0, phi^1_10],
[phi^1_10,        0]])

In [16]:
H0 = Hs + H1 + H2
V = H_int1 + H_int2

In [17]:
import sympy as sp

def shuffle_H0_and_V(H0, V, d):
    # Make H0 mutable to allow modifications
    H0_new = H0.as_mutable()
    
    # Make V mutable to allow modifications
    V_new = V.as_mutable()

        
    # Swap diagonal elements at indices 3 and d+1
    H0_new[3, 3], H0_new[d+1, d+1] = H0[d+1, d+1], H0[3, 3]
    
    # Swap rows and columns at indices 3 and d+1 in V
    V_new[3, :], V_new[d+1, :] = V[d+1, :], V[3, :]
    V_new[:, 3], V_new[:, d+1] = V[:, d+1], V[:, 3]
    
    # Swap diagonal elements at indices 2 and d
    H0_new[2, 2], H0_new[d, d] = H0[d, d], H0[2, 2]
    
    # Swap rows and columns at indices 2 and d in V
    V_new[2, :], V_new[d, :] = V_new[d, :], V_new[2, :]
    V_new[:, 2], V_new[:, d] = V_new[:, d], V_new[:, 2]

    
    # Optionally convert back to immutable matrices if needed
    H0_new = H0_new.as_immutable()
    V_new = V_new.as_immutable()

    return H0_new, V_new
import sympy as sp

def commutator(A, B):
    return A * B - B * A

def nested_commutator(A, B, n):
    if n == 1:
        return commutator(A, B)
    else:
        return commutator(A, nested_commutator(A, B, n-1))

def create_subspace_projector(dim, subspace_indices):
    """
    Create a projector for the given subspace using SymPy.

    Args:
        dim (int): The total dimension of the Hilbert space.
        subspace_indices (list): The indices of the subspace.

    Returns:
        tuple: The projector P for the given subspace and the complementary projector Q (both SymPy matrices).
    """
    # Initialize the projector matrix P as a zero matrix of size dim x dim
    P = sp.zeros(dim, dim)
    
    # Create the basis states and compute the projector for each
    for idx in subspace_indices:
        # Create a basis state (column vector) with 1 at the index 'idx'
        state = sp.zeros(dim, 1)
        state[idx, 0] = 1
        
        # Compute the rank-1 projector for this basis state: |state><state|
        P += state * state.T  # Outer product in SymPy is `*`
    # The complementary projector Q = I - P
    Q = sp.eye(dim) - P
    
    return P, Q

def swt_subspace(H0, V, subspace_indices):
    """
    Compute the Schrieffer-Wolff transformation on a subspace of the Hamiltonian.

    Args:
        H0 (sp.Matrix): The unperturbed Hamiltonian.
        V (sp.Matrix): The perturbation Hamiltonian.
        subspace_indices (list): The indices of the subspace to consider.

    Returns:
        tuple: The Schrieffer-Wolff transformation operators (S1, S2, S3) and 
               the transformed Hamiltonian components (H1, H2, H3, H4).
    """
    dim = H0.shape[0]
    P, Q = create_subspace_projector(dim, subspace_indices)
    
    # Extract the subspace of H0 and V
    Vd = P * V * P + Q * V * Q
    Vod = P * V * Q + Q * V * P

    # Compute the energy differences in the subspace
    delta = sp.Matrix([[H0[i,i] - H0[j,j] if i != j else 1 for j in range(dim)] for i in range(dim)])

    # Compute the Schrieffer-Wolff transformation components
    H1 = Vd
    S1 = sp.Matrix([[Vod[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])
    H2 = sp.Rational(1, 2) * commutator(S1, Vod)
    S2 = sp.Matrix([[-commutator(Vd, S1)[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])

    # H3 = sp.Rational(1, 2) * commutator(S2, Vod)
    # S3 = sp.Matrix([[(commutator(S2, Vd)[i,j] + sp.Rational(1, 3) * nested_commutator(S1, Vod, 2)[i,j]) / delta[i,j] for j in range(dim)] for i in range(dim)])
    # H4 = sp.Rational(1, 2) * commutator(S3, Vod) - sp.Rational(1, 24) * nested_commutator(S1, Vod, 3)

    return [S1,S2], [H1, H2,]


In [18]:
H0,V = shuffle_H0_and_V(H0, V, d)

In [19]:
indices = [0,1,2,3]
result = swt_subspace(H0, V, indices)

In [20]:
S1,S2 = result[0]

In [21]:
(result[1][1])[indices, :][:, indices]

Matrix([
[                                             -E_L**2*g3**2*phi^1_10**2*phi_s**2/(omega**2*(alpha + epsilon^1_1 + 2*omega_s)) + E_L**2*g3**2*phi^1_10**2*phi_s**2/(omega**2*(-alpha - epsilon^1_1 - 2*omega_s)) - E_L**2*g3**2*phi^2_10**2*phi_s**2/(omega**2*(alpha + epsilon^2_1 + 2*omega_s)) + E_L**2*g3**2*phi^2_10**2*phi_s**2/(omega**2*(-alpha - epsilon^2_1 - 2*omega_s)) - E_L**2*phi^1_10**2*phi_s**2/(8*(epsilon^1_1 + omega_s)) + E_L**2*phi^1_10**2*phi_s**2/(8*(-epsilon^1_1 - omega_s)) - E_L**2*phi^2_10**2*phi_s**2/(8*(epsilon^2_1 + omega_s)) + E_L**2*phi^2_10**2*phi_s**2/(8*(-epsilon^2_1 - omega_s)),                                                                                                                                                                                                                                                                                                                                                                                                  

In [23]:
chargeop = -sp.Rational(1,2)*(s-s_dagger)/(phis)*I
chargeop = TensorProduct(chargeop,TensorProduct(I2,I1))

In [24]:
chargeop

Matrix([
[          0,           0,           0,           0,        -I/(2*phi_s),                   0,                   0,                   0,                    0,                    0,                    0,                    0,                    0,                    0,                    0,                    0,                   0,                   0,                   0,                   0,                    0,                    0,                    0,                    0],
[          0,           0,           0,           0,                   0,        -I/(2*phi_s),                   0,                   0,                    0,                    0,                    0,                    0,                    0,                    0,                    0,                    0,                   0,                   0,                   0,                   0,                    0,                    0,                    0,                    0],
[          0,      

In [25]:

# Define the commutator function
def comm(X, Y):
    return X*Y - Y*X

In [26]:
x= comm(S2, chargeop)[0,3]

In [27]:
x

I*(-E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(alpha - epsilon^2_1 + 2*omega_s)) - E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(alpha - epsilon^1_1 + 2*omega_s)) + 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 + omega_s)) + 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^1_1 + omega_s)))/(2*phi_s*(-epsilon^1_1 - epsilon^2_1 + omega_s)) + I*(E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-alpha - epsilon^2_1 - 2*omega_s)) + E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-alpha - epsilon^1_1 - 2*omega_s)) - 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 - omega_s)) - 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^1_1 - omega_s)))/(2*phi_s*(-epsilon^1_1 - epsilon^2_1 - omega_s))

In [28]:
y = comm(S2, chargeop)[1,2]

In [29]:
y

I*(-E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(alpha - epsilon^2_1 + 2*omega_s)) - E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(alpha + epsilon^1_1 + 2*omega_s)) + 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 + omega_s)) + 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(epsilon^1_1 + omega_s)))/(2*phi_s*(epsilon^1_1 - epsilon^2_1 + omega_s)) + I*(E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-alpha - epsilon^2_1 - 2*omega_s)) + E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-alpha + epsilon^1_1 - 2*omega_s)) - 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 - omega_s)) - 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(epsilon^1_1 - omega_s)))/(2*phi_s*(epsilon^1_1 - epsilon^2_1 - omega_s))

In [30]:
# from sympy import expand, collect
# # Define symbolic variables and operators
# lambda_sym = sp.Symbol('lambda')
# # A = sp.Symbol('A', commutative=False)
# # S1 = sp.Symbol('S1', commutative=False)
# # S2 = sp.Symbol('S2', commutative=False)
# # S3 = sp.Symbol('S3', commutative=False)

# # Define S up to third order in lambda
# S = lambda_sym * S1 + lambda_sym**2 * S2 
# # + lambda_sym**3 * S3


# # Perform BCH expansion
# result = chargeop + comm(S, chargeop) + (1/2) * comm(S, comm(S, chargeop))
# # + (1/6) * comm(S, comm(S, comm(S, A)))

# # Expand the result
# expanded_result = expand(result)

# # Collect terms by order of lambda
# collected = collect(expanded_result, lambda_sym, evaluate=False)

# first_order = collected.get(lambda_sym, 0)
# second_order = collected.get(lambda_sym**2, 0)
# # third_order = collected.get(lambda_sym**3, 0)
    

In [31]:
# second_order

In [32]:
chargeop_trans = sp.Rational(1,2)*chargeop*S1@S1 - chargeop*S2 - S1*chargeop*S1 + sp.Rational(1,2)*S1*S1*chargeop + S2*chargeop

In [33]:
epsilon1 = [0] + [sp.Symbol(f'epsilon^1_{i}') for i in range(1, d)]
epsilon2 = [0] + [sp.Symbol(f'epsilon^2_{i}') for i in range(1, d)]

In [34]:
epsilon1[1],epsilon2[1]

(epsilon^1_1, epsilon^2_1)

In [35]:
# Create symbolic variables for epsilon1 and epsilon2, but set the first element to 0
x = chargeop_trans[0,3].subs(alpha,0)

In [36]:
y = chargeop_trans[1,2].subs(alpha,0)

In [37]:
x

I*E_L**2*g3*phi^1_10*phi^2_10*phi_s/(2*omega*(-epsilon^1_1 + 2*omega_s)*(-epsilon^2_1 - omega_s)) - I*E_L**2*g3*phi^1_10*phi^2_10*phi_s/(2*omega*(-epsilon^1_1 + omega_s)*(-epsilon^2_1 - 2*omega_s)) + I*E_L**2*g3*phi^1_10*phi^2_10*phi_s/(2*omega*(-epsilon^1_1 - omega_s)*(-epsilon^2_1 + 2*omega_s)) - I*E_L**2*g3*phi^1_10*phi^2_10*phi_s/(2*omega*(-epsilon^1_1 - 2*omega_s)*(-epsilon^2_1 + omega_s)) + I*(-E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 + 2*omega_s)) + 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 + omega_s)) - E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^1_1 + 2*omega_s)) + 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^1_1 + omega_s)))/(2*phi_s*(-epsilon^1_1 - epsilon^2_1 + omega_s)) + I*(-3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 - omega_s)) + E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^2_1 - 2*omega_s)) - 3*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/(omega*(-epsilon^1_1 - omega_s)) + E_L**2*g3*phi^1_10*phi^2

In [38]:
expansion_x = x.series(epsilon1[1]/omega_s, 0, 2).removeO()  # 2nd order expansion

expansion_y = y.series(epsilon2[1]/omega_s, 0, 2).removeO()

In [52]:
(x).subs(epsilon1[1],0.41).subs(epsilon2[1],0.6).subs(omega_s,5)

0.068135016066278*I*E_L**2*g3*phi^1_10*phi^2_10*phi_s/omega

In [63]:
(y).subs(epsilon1[1],0.041).subs(omega_s,3.1).subs(epsilon2[1],0.06)

0.00510516094489746*I*E_L**2*g3*phi^1_10*phi^2_10*phi_s/omega

In [41]:
(x.args[0]+x.args[1]).args[0].args[4].subs(epsilon1[1],0.41).subs(omega_s,3.1).subs(epsilon2[1],0.6)

1.963958635441*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/omega

In [42]:
(x.args[0]+x.args[1]).args[1].args[4].subs(epsilon1[1],0.41).subs(omega_s,3.1).subs(epsilon2[1],0.6)

1.36716691157378*E_L**2*g3*phi^1_10*phi^2_10*phi_s**2/omega

In [233]:
S1

Matrix([
[                                                                      0,                                                                       0,                                                                       0,                                                                       0,                                              0, E_L*phi^1_10*phi_s/(2*(-epsilon^1_1 - omega_s)), E_L*phi^2_10*phi_s/(2*(-epsilon^2_1 - omega_s)),                                               0,                                                                        0, sqrt(2)*E_L*g3*phi^1_10*phi_s/(omega*(-alpha - epsilon^1_1 - 2*omega_s)), sqrt(2)*E_L*g3*phi^2_10*phi_s/(omega*(-alpha - epsilon^2_1 - 2*omega_s)),                                                                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[                                                                      0,                                                                       0,                        

In [234]:
S2

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    0,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        